In [ ]:
# OBJECTIVE: Mark launch sites on a map using folium and include info such as success/failed launches, distance between 
# launch sites and various locations to find geographical patterns about launch sites

In [3]:
import folium
import wget
import pandas as pd

In [4]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

from math import sin, cos, sqrt, atan2, radians

In [5]:
# Download and read `spacex_launch_geo.csv` provided by Coursera. It is an augmented dataset with latitude and longitude info
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
# Select relevant sub-columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Create new dataframe for launch sites and their coordinates
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circle object based on its coordinate for each launch site. Add Launch site name as a popup label
launch_sites = list(launch_sites_df['Launch Site'])
lat = list(launch_sites_df['Lat'])
long = list(launch_sites_df['Long'])
                    
for i in range(len(launch_sites)):
    coor = [lat[i], long[i]]
    circle = folium.Circle(coor, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_sites[i]))
    marker = folium.map.Marker(coor, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites[i]))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

In [12]:
# Create new column 'marker_color' for success/fail based on 'class'
spacex_df['marker_color'] = ['green' if val == 1 else 'red' for val in spacex_df['class']]

spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [13]:
# Add a marker to marker_cluster for each launch result

marker_clr = list(spacex_df['marker_color'])
lati = list(spacex_df['Lat'])
longi = list(spacex_df['Long'])
sites = list(spacex_df['Launch Site'])

for i in range(len(marker_clr)):
    coordi = [lati[i], longi[i]]
    marker = folium.Marker(coordi,popup=sites[i], icon=folium.Icon(color=marker_clr[i]))
    site_map.add_child(marker)
    marker_cluster.add_child(marker)

site_map

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
# Function that calculate distance between two sets of coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# find coordinate of the closet coastline
coastline_lat_cali = 34.63575
coastline_lon_cali = -120.62606

# e.g.,: Lat: 28.56367  Lon: -80.57163
distance_coastline_cali = calculate_distance(lat[3], long[3], coastline_lat_cali, coastline_lon_cali)

distance_coastline_cali

1.438611325187494

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker_coast_cali = folium.Marker(
   [coastline_lat_cali, coastline_lon_cali],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_cali),
       )
   )

site_map.add_child(distance_marker_coast_cali)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [43]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[[coastline_lat_cali, coastline_lon_cali], [lat[3], long[3]]], color='red', weight=1)
site_map.add_child(lines)


In [44]:
# Get all nearest coastline, rail, highway, and city coordinates and place in lists
# Create empty lists to store distance coordinates

coastline_lat = [28.56276, 28.56276, 28.5809, 34.63575]
coastline_lon = [-80.56736, -80.56736, -80.5727, -120.62606]
coastline_dis = []

rail_lat = [28.5719, 28.5719, 28.57344, 34.63504]
rail_lon = [-80.58525, -80.58525, -80.65411, -120.62442]
rail_dis = []

highway_lat = [28.56293, 28.56293, 28.57307, 34.64105]
highway_lon = [-80.57075, -80.57075, -80.65551, -120.43578]
highway_dis = []

city_lat = [28.10893, 28.10893, 28.10893, 34.63787]
city_lon = [-80.64445, -80.64445, -80.64445, -120.45771]
city_dis = []

launch_sites_df['coastline_lat'] = coastline_lat
launch_sites_df['coastline_lon'] = coastline_lon
launch_sites_df['rail_lat'] = rail_lat
launch_sites_df['rail_lon'] = rail_lon
launch_sites_df['highway_lat'] = highway_lat
launch_sites_df['highway_lon'] = highway_lon
launch_sites_df['city_lat'] = city_lat
launch_sites_df['city_lon'] = city_lon

for i in range(4):
    # Calculate distance using above coordinate lists and append to empty distance lists
    coast_d = calculate_distance(lat[i], long[i], coastline_lat[i], coastline_lon[i])
    coastline_dis.append(coast_d)
    
    rail_d = calculate_distance(lat[i], long[i], rail_lat[i], rail_lon[i])
    rail_dis.append(rail_d)
    
    highway_d = calculate_distance(lat[i], long[i], highway_lat[i], highway_lon[i])
    highway_dis.append(highway_d)
    
    city_d = calculate_distance(lat[i], long[i], city_lat[i], city_lon[i])
    city_dis.append(city_d)
    
    
    # Draw lines on map between launch site and its nearest 4 types of locations
    lines_coast=folium.PolyLine(locations=[[lat[i], long[i]], [coastline_lat[i], coastline_lon[i]]], color='red', weight=1)
    site_map.add_child(lines_coast)
    
    lines_rail=folium.PolyLine(locations=[[lat[i], long[i]], [rail_lat[i], rail_lon[i]]], color='black', weight=1)
    site_map.add_child(lines_rail)
    
    lines_highway=folium.PolyLine(locations=[[lat[i], long[i]], [highway_lat[i], highway_lon[i]]], color='green', weight=1)
    site_map.add_child(lines_highway)
    
    lines_city=folium.PolyLine(locations=[[lat[i], long[i]], [city_lat[i], city_lon[i]]], color='purple', weight=1)
    site_map.add_child(lines_city)
    
    
    # Mark distance value on map between launch site and its nearest 4 types of locations
    distance_marker_coast = folium.Marker([coastline_lat[i], coastline_lon[i]], 
                                           icon=DivIcon(icon_size=(20,20),
                                                        icon_anchor=(0,0),
                                                        html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(coast_d))).add_to(site_map)
    
    distance_marker_rail = folium.Marker([rail_lat[i], rail_lon[i]], 
                                           icon=DivIcon(icon_size=(20,20),
                                                        icon_anchor=(0,0),
                                                        html='<div style="font-size: 12; color:black;"><b>%s</b></div>' % "{:10.2f} KM".format(rail_d))).add_to(site_map)
    
    distance_marker_highway = folium.Marker([highway_lat[i], highway_lon[i]], 
                                           icon=DivIcon(icon_size=(20,20),
                                                        icon_anchor=(0,0),
                                                        html='<div style="font-size: 12; color:green;"><b>%s</b></div>' % "{:10.2f} KM".format(highway_d))).add_to(site_map)
    
    distance_marker_city = folium.Marker([city_lat[i], city_lon[i]], 
                                           icon=DivIcon(icon_size=(20,20),
                                                        icon_anchor=(0,0),
                                                        html='<div style="font-size: 12; color:purple;"><b>%s</b></div>' % "{:10.2f} KM".format(city_d))).add_to(site_map)

    
# Add new columns to dataframe based on calculated distances for each location    
launch_sites_df['coastline_dis'] = coastline_dis
launch_sites_df['rail_dis'] = rail_dis
launch_sites_df['highway_dis'] = highway_dis
launch_sites_df['city_dis'] = city_dis


site_map

In [45]:
launch_sites_df

,Launch Site,Lat,Long,coastline_lat,coastline_lon,rail_lat,rail_lon,highway_lat,highway_lon,city_lat,city_lon,coastline_dis,rail_dis,highway_dis,city_dis
0,CCAFS LC-40,28.562302,-80.577356,28.56276,-80.56736,28.57190,-80.58525,28.56293,-80.57075,28.10893,-80.64445,0.977911,1.316946,0.649174,50.854487
1,CCAFS SLC-40,28.563197,-80.576820,28.56276,-80.56736,28.57190,-80.58525,28.56293,-80.57075,28.10893,-80.64445,0.925450,1.270911,0.593739,50.960023
2,KSC LC-39A,28.573255,-80.646895,28.58090,-80.57270,28.57344,-80.65411,28.57307,-80.65551,28.10893,-80.64445,7.297058,0.705054,0.841760,51.647299
3,VAFB SLC-4E,34.632834,-120.610745,34.63575,-120.62606,34.63504,-120.62442,34.64105,-120.43578,34.63787,-120.45771,1.438611,1.275313,16.038298,14.016749


Copyright © 2021 IBM Corporation. All rights reserved.
